## 📚 Exercise 1
In this exercise, we will understand the functioning of TF/IDF ranking and implement and test a vector space retrieval model.

### Use case:
You are an engineer who is building a search engine for the recipe finder application _bestbakesinthebuilding.com_. In this application, users can search for their favorite recipe, ingredients, or any recipe-relevant term. The search engine should return relevant recipes that are stored in the website's DB. For the implementation of the searching functionality, you have to use the vector space retrieval model.

To build your system, you have available recipe data in the file bread.txt which is a dump from the application's DB.

The schema of the data is the following:

  DocID | Document Text
  ------|------------------
  1     | How to Bake Breads Without Baking Recipes
  2     | Smith Pies: Best Pies in London
  3     | Numerical Recipes: The Art of Scientific Computing
  4     | Breads, Pastries, Pies, and Cakes: Quantity Baking Recipes
  5     | Pastry: A Book of Best French Pastry Recipes


### Goal:
1. Preprocess the input text by tokenizing it and stemming it.
2. Vectorize the preprocessed text by implementing the TF/IDF function.
3. Test your method by searching for the query $Q = ``baking''$ and find the top-ranked documents according to the TF/IDF rank. 
4. Compare your implementation with the existing implementation of scikit-learn library.

### What are you learning in this exercise:
- Vector space retrieval model implementation.
- TF/IDF ranking.
- Evaluation of a text retrieval system.


In [1]:
# Loading of libraries and documents
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
from nltk.corpus import stopwords
import math
from collections import Counter
from operator import itemgetter
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/aibin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/aibin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### 1. Preprocess the input text by tokenizing it and stemming it.

In [56]:
stemmer = PorterStemmer() 
def tokenize(text):
    """
    It tokenizes and stems an input text.
    
    :param text: str, with the input text
    :return: list, of the tokenized and stemmed tokens.
    """
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    return [stemmer.stem(word.lower()) for word in tokens if word not in stopwords.words('english')]

# Read a list of documents from a file. Each line in a file is a document
with open("epfldocs.txt") as f:
    content = f.readlines()
original_documents = [x.strip() for x in content]
documents = [tokenize(d) for d in original_documents]

# documents

In [67]:
test = "Numerical Recipes: The Art of Scientific Computing"
test_punc = "".join([ch for ch in test if ch not in string.punctuation])
test_punc

documents
test_vocab = [vocab for sentence in documents for vocab in sentence]
test_vocab

len(documents)


test_term = 'recip'
test_term_doc = [test_term in document for document in documents]
test_term_doc_num = sum([test_term in document for document in documents])
test_term_doc

test_vector = [0]*6
test_vector

test_counts = Counter(documents[0])
max_count = test_counts.most_common(1)[0]
test_counts

# vector[i] =  sum([term in document]) * idf[term] / max_count  FALSE FALSE FALSE FALSE FALSE
sum(['bake' in documents[0]])
[term == 'bake' for term in documents[0]]

txt = "welcome to the jungle"
txt_split = txt.split() # cannot remove punctuation
txt_split[:2] # exclude the last one

['welcome', 'to']

#### 2. Vectorize the preprocessed text by implementing the TF/IDF function.

In [59]:
# Functions for the TF/IDF implementation

# create the vocabulary
vocabulary = list(set([item for sublist in documents for item in sublist]))
vocabulary.sort()

def idf_values(vocabulary, documents):
    """
    It computes IDF scores, storing idf values in a dictionary.
    
    :param vocabulary: list of str, with the unique tokens of the vocabulary.
    :param documents: list of lists of str, with tokenized sentences.
    :return: dict with the idf values for each vocabulary word.
    """
    idf = {}
    num_documents = len(documents)
    for i, term in enumerate(vocabulary):
        term_doc_num = sum([term in document for document in documents])
        idf[term] = math.log(num_documents/term_doc_num)
    return idf

def vectorize(document, vocabulary, idf):
    """
    It generates the vector for an input document (with normalization).
    
    :param document: list of str with the tokenized documents.
    :param vocabulary: list of str, with the unique tokens of the vocabulary.
    :param idf: dict with the idf values for each vocabulary word.
    :return: list of floats
    """
    vector = [0]*len(vocabulary)
    counts = Counter(document)
    max_count = counts.most_common(1)[0][1]
    for i,term in enumerate(vocabulary):
        # YOUR CODE HERE
        vector[i] =  counts[term] * idf[term] / max_count
    return vector

# Compute IDF values and vectors
idf = idf_values(vocabulary, documents)
document_vectors = [vectorize(s, vocabulary, idf) for s in documents]

def cosine_similarity(v1,v2):
    """
    It computes cosine similarity.
    
    :param v1: list of floats, with the vector of a document.
    :param v2: list of floats, with the vector of a document.
    :return: float
    """
    
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    if sumxy == 0:
            result = 0
    else:
        # YOUR CODE HERE
            result = sumxy / math.sqrt(sumxx * sumyy)
    return result

def search_vec(query, topk):
    """
    It computes the search result (get the topk documents).
    
    :param query: str
    :param topk: int
    """
    q = query.split()
    q = [stemmer.stem(w) for w in q]
    query_vector = vectorize(q, vocabulary, idf)
    scores = [[cosine_similarity(query_vector, document_vectors[d]), d] for d in range(len(documents))]
    scores.sort(key=lambda x: -x[0])
    for i in range(topk):
        print(original_documents[scores[i][1]])

# HINTS
# natural logarithm function
#     math.log(n,math.e)
# Function to count term frequencies in a document
#     Counter(document)
# most common elements for a list
#     counts.most_common(1)

In [58]:
range(len(documents))

range(0, 1075)

#### 3. Test your method by searching for the query  𝑄=‘‘𝑏𝑎𝑘𝑖𝑛𝑔″  and find the top-ranked documents according to the TF/IDF rank.

In [44]:
# Reference code using scikit-learn
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 1, stop_words = 'english')
features = tf.fit_transform(original_documents)
npm_tfidf = features.todense()
new_features = tf.transform(['baking'])

cosine_similarities = linear_kernel(new_features, features).flatten()
related_docs_indices = cosine_similarities.argsort()[::-1]
topk = 5
for i in range(topk):
    print(original_documents[related_docs_indices[i]])

How to Bake Breads Without Baking Recipes
Breads, Pastries, Pies, and Cakes: Quantity Baking Recipes
Pastry: A Book of Best French Pastry Recipes
Numerical Recipes: The Art of Scientific Computing
Smith Pies: Best Pies in London


In [55]:
topk = 5
search_vec('baking', topk)

How to Bake Breads Without Baking Recipes
Breads, Pastries, Pies, and Cakes: Quantity Baking Recipes
Smith Pies: Best Pies in London
Numerical Recipes: The Art of Scientific Computing
Pastry: A Book of Best French Pastry Recipes


#### 4.Compare your implementation with the existing implementation of scikit-learn library.

In this part, we consider the scikit reference code as an “oracle” that supposedly gives the correct result. Your task is to compare your implemented tf-idf retrieval model with this oracle for the following queries "computer science", "IC school", "information systems" on the **epfldocs.txt** collection.

For this exercise, you need to replace **bread.txt** in the first cell with **epfldocs.txt** and rerun all the cells from the begining. 

In [49]:
# Retrieval oracle 
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 1, stop_words = 'english')
features = tf.fit_transform(original_documents)
npm_tfidf = features.todense()

def search_vec_sklearn(query, features, threshold=0.1):
    """
    Return all document ids that that have cosine similarity with the query larger than a threshold.
    
    :param query: str
    :param features: ndarray
    :param threshold: float
    :return: list of int
    """
    new_features = tf.transform([query])
    cosine_similarities = linear_kernel(new_features, features).flatten()
    related_docs_indices, cos_sim_sorted = zip(*sorted(enumerate(cosine_similarities), key=itemgetter(1), 
                                                       reverse=True))
    doc_ids = []
    for i, cos_sim in enumerate(cos_sim_sorted):
        if cos_sim < threshold:
            break
        doc_ids.append(related_docs_indices[i])
    return doc_ids

In [50]:
ret_ids = search_vec_sklearn('computer science', features)
for i, v in enumerate(ret_ids):
    print(original_documents[v])

Exciting News: "World University Rankings 2016-2017 by subject: computer science" No1 @ETH &amp; @EPFL on No8. Congrats https://t.co/ARSlXZoShQ
New computer model shows how proteins are controlled "at a distance" https://t.co/zNjK3bZ6mO  via @EPFL_en #VDtech https://t.co/b9TglXO4KD
An interview with Patrick Barth, a new @EPFL professor who combines protein #biophysics with computer modeling https://t.co/iJwBaEbocj
Exposure Science Film Hackathon 2017 applications open! Come join our Scicomm-film-hacking event! #Science #scicomm https://t.co/zwtKPlh6HT
Le mystère Soulages éblouit la science @EPFL  https://t.co/u3uNICyAdi
@cwarwarrior @EPFL_en @EPFL Doing science at @EPFL_en is indeed pretty cool!!! Thank you for visiting!!!
Blue Brain Nexus: an open-source tool for data-driven science https://t.co/m5yTgXf7ym #epfl
Swiss Data Science on Twitter: "Sign up for @EPFL_en #DataJamDays: learn more a… https://t.co/kNVILHWPGb, see more https://t.co/2wg3BbHBNq
The registration for Exposure Scienc

In [ ]:
# modified version
def search_vec(query, topk):
    """
    It computes the search result (get the topk documents).
    
    :param query: str
    :param topk: int
    """
    q = query.split()
    q = [stemmer.stem(w) for w in q]
    query_vector = vectorize(q, vocabulary, idf)
    scores = [[cosine_similarity(query_vector, document_vectors[d]), d] for d in range(len(documents))]
    scores.sort(key=lambda x: -x[0])
    for i in range(topk):
        print(original_documents[scores[i][1]])

In [60]:
search_vec('computer science', 10)

Exciting News: "World University Rankings 2016-2017 by subject: computer science" No1 @ETH &amp; @EPFL on No8. Congrats https://t.co/ARSlXZoShQ
New computer model shows how proteins are controlled "at a distance" https://t.co/zNjK3bZ6mO  via @EPFL_en #VDtech https://t.co/b9TglXO4KD
An interview with Patrick Barth, a new @EPFL professor who combines protein #biophysics with computer modeling https://t.co/iJwBaEbocj
Video of Nicola Marzari from @EPFL_en  on Computational Discovery in the 21st Century during #PASC17 now online: https://t.co/tfCkEvYKtq https://t.co/httPdHcK9W
New at @epfl_en Life Sciences @epflSV: "From PhD directly to Independent Group Leader" #ELFIR_EPFL:  Early Independence Research Scholars. See https://t.co/evqyqD7FFl, also for computational biology #compbio https://t.co/e3pDCg6NVb Deadline April 1 2018 at https://t.co/mJqcrfIqkb
@CodeWeekEU is turning 5, yay! We look very much forward to computational thinking unplugged activities during @CodeWeek_CH https://t.co/yDP

In [ ]:
queries = ["computer science", "IC school", "information systems"]

In [ ]:
def compute_recall_at_k(predict, gt, k):
    """
    It computes the recall score at a defined set of retrieved documents.
    
    :param predict: list of predictions
    :param gt: list of actual data
    :param k: int
    """
    # YOUR CODE HERE
    correct = set(predict[:k]).intersection(set(gt))
    return len(correct) / len(gt)

In [ ]:
def compute_precision_at_k(predict, gt, k):
    """
    It computes the precision score at a defined set of retrieved documents.
    
    :param predict: list of predictions
    :param gt: list of actual data
    :param k: int
    """
    # YOUR CODE HERE
    correct = set(predict[:k]).intersection(set(gt))
    return len(correct) / k

In [ ]:
def compute_map(queries, K = 10):
    """
    It computes mean average precision (MAP) for a set of queries.
    
    :param queries: list of str
    :param K: int
    """
    map_score = 0
    for i, query in enumerate(queries):
        precision_for_query = 0
        predict = search_vec(query, K)
        gt = search_vec_sklearn(query, features)
        for k in range(1, K+1):
            # YOUR CODE HERE
        # YOUR CODE HERE
        map_score += 
    # YOUR CODE HERE
    map_score = map_score / len(queries) 
    return map_score

In [ ]:
compute_map(queries)